# Imports

In [70]:
import pandas as pd
import json
import datetime

## Helper Functions

# Load Data

In [71]:
# Opening JSON file
train = open('dataset/raw/train.json')
 
# returns JSON object as 
# a dictionary
data_train_aux = json.load(train)

data_train = pd.json_normalize(data_train_aux, record_path = 'data')
data_train.columns = data_train_aux['columns']
data_train.head()

,_id,average_rating,number_of_reviews,brand,category,crawled_at,description,images,out_of_stock,avg_delivery_time_days,pid,product_details,seller,sub_category,fabrication_time,title,actual_price
0,53df9662-e500-569c-946e-0c8d215a72cd,3.2,26,East I,Clothing and Accessories,2021-02-10 21:17:28,Navy Blue Printed Boxers Has An Inner Elasti...,['https://rukminim1.flixcart.com/image/128/128...,False,8,BXRFTZF7JGX75DAW,"[{'Color': 'Dark Blue'}, {'Fabric': 'Pure Cott...",ZIYAA,Innerwear and Swimwear,653,Printed Men Boxer (Pack of 1),849.0
1,d0142842-84f7-537d-a06f-d85b76488a5f,4.0,33,dream o,Clothing and Accessories,2021-02-11 01:02:46,smiley printed tshirt on round neck cotton tshirt,['https://rukminim1.flixcart.com/image/128/128...,False,12,TSHFWQM96UHR6A4Q,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Dream Onn Creations,Topwear,668,Printed Men Round Neck Orange T-Shirt,699.0
2,79c8f0d7-30b1-5dd4-9f2f-2fe97782b027,3.9,32,Free Authori,Clothing and Accessories,2021-02-11 00:43:37,Free Authority Presents this Crew Neck Yellow ...,['https://rukminim1.flixcart.com/image/128/128...,False,11,SWSFWCXH2WF6ZYRB,"[{'Color': 'Yellow'}, {'Fabric': 'Polycotton'}...",BioworldMerchandising,Winter Wear,53,Full Sleeve Graphic Print Men Sweatshirt,1499.0
3,0531c28c-7c50-5fbd-9ce3-a7cae3243ad5,3.8,31,HUMBE,Clothing and Accessories,2021-02-10 21:22:10,Cotton Blend FabricCollar / Polo Neck White & ...,['https://rukminim1.flixcart.com/image/128/128...,False,11,TSHFHQH3HKDAGGK9,"[{'Type': 'Polo Neck'}, {'Sleeve': 'Short Slee...",HUMBERT,Topwear,510,"Solid Men Polo Neck Light Blue, White T-Shirt ...",1699.0
4,d604baad-472e-5c18-86a3-7b46d4a890c2,2.4,20,Rose We,Clothing and Accessories,2021-02-10 23:36:36,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,5,TSHFW9CJZSYUU6UX,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Rupalcollectionjaipur,Topwear,496,Printed Men Round Neck White T-Shirt,599.0


# Data Description

In [72]:
# retirar descrição
# retirar imagem
# verificar ids duplicados em id produto
# Verificar outliers no price

## Removendo colunas

In [73]:
data_train = data_train.drop(columns=['description', 'images'])

## Transformando tipo de dados

In [74]:
data_train.dtypes

_id                        object
average_rating            float64
number_of_reviews           int64
brand                      object
category                   object
crawled_at                 object
out_of_stock                 bool
avg_delivery_time_days      int64
pid                        object
product_details            object
seller                     object
sub_category               object
fabrication_time            int64
title                      object
actual_price              float64
dtype: object

In [75]:
# transformando de string para data
data_train['crawled_at'] = pd.to_datetime(data_train['crawled_at']).dt.date

## Tratando valores nulos

In [76]:
data_train.isna().sum()/data_train.shape[0]

_id                       0.000000
average_rating            0.000000
number_of_reviews         0.000000
brand                     0.000000
category                  0.000000
crawled_at                0.000000
out_of_stock              0.000000
avg_delivery_time_days    0.000000
pid                       0.000000
product_details           0.000000
seller                    0.000000
sub_category              0.000000
fabrication_time          0.000000
title                     0.000000
actual_price              0.149985
dtype: float64

In [77]:
data_train = data_train.dropna(subset=['actual_price'])

In [78]:
data_train.isna().sum()/data_train.shape[0]

_id                       0.0
average_rating            0.0
number_of_reviews         0.0
brand                     0.0
category                  0.0
crawled_at                0.0
out_of_stock              0.0
avg_delivery_time_days    0.0
pid                       0.0
product_details           0.0
seller                    0.0
sub_category              0.0
fabrication_time          0.0
title                     0.0
actual_price              0.0
dtype: float64

## Entendendo prodct Details

In [79]:
import ast

# pega string de dict e cria uma lista de dicionários
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []

teste = data_train['product_details'].apply(lambda x: f(x))

def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario


df = pd.DataFrame([junta_dict(row) for row in teste])

In [83]:
df = pd.DataFrame([junta_dict(row) for row in teste])
df.isna().sum().sort_values()

Fabric               1491
Style Code           1653
Pattern              1661
Fabric Care          4305
Suitable For         6120
                    ...  
Shoulder in inch    19811
Pleated             19812
Region              19812
Design              19812
Fabric care         19812
Length: 124, dtype: int64

# Feature Engineering

In [62]:
df4 = data_train.copy()

In [63]:
df4.head()

,_id,average_rating,number_of_reviews,brand,category,crawled_at,out_of_stock,avg_delivery_time_days,pid,seller,sub_category,fabrication_time,title,actual_price
0,53df9662-e500-569c-946e-0c8d215a72cd,3.2,26,East I,Clothing and Accessories,2021-02-10,False,8,BXRFTZF7JGX75DAW,ZIYAA,Innerwear and Swimwear,653,Printed Men Boxer (Pack of 1),849.0
1,d0142842-84f7-537d-a06f-d85b76488a5f,4.0,33,dream o,Clothing and Accessories,2021-02-11,False,12,TSHFWQM96UHR6A4Q,Dream Onn Creations,Topwear,668,Printed Men Round Neck Orange T-Shirt,699.0
2,79c8f0d7-30b1-5dd4-9f2f-2fe97782b027,3.9,32,Free Authori,Clothing and Accessories,2021-02-11,False,11,SWSFWCXH2WF6ZYRB,BioworldMerchandising,Winter Wear,53,Full Sleeve Graphic Print Men Sweatshirt,1499.0
3,0531c28c-7c50-5fbd-9ce3-a7cae3243ad5,3.8,31,HUMBE,Clothing and Accessories,2021-02-10,False,11,TSHFHQH3HKDAGGK9,HUMBERT,Topwear,510,"Solid Men Polo Neck Light Blue, White T-Shirt ...",1699.0
4,d604baad-472e-5c18-86a3-7b46d4a890c2,2.4,20,Rose We,Clothing and Accessories,2021-02-10,False,5,TSHFW9CJZSYUU6UX,Rupalcollectionjaipur,Topwear,496,Printed Men Round Neck White T-Shirt,599.0


In [66]:
## out_of_stock - transformar para int 
df4['out_of_stock'] = df4['out_of_stock'].astype('int64')

# brand - substituir por outros

# EDA

# Data Preparation

In [68]:
'_id'
'average_rating':         muitos outliers mas distribuição proxima da normal
                          tratar notas que não possuem avaliação
'number_of_reviews':      muitos outliers mas distribuição proxima da normal
                          ver como fica a distribuição como log
'brand':                  12 categorias frequency -> transformar tudo em minusculo para evitar duplicações
'category':               Uma categoria é mais majoritária 
                          unir com a subcategory fazendo o tratamento de maiusculas
'crawled_at':             Somente 1 dia, muda somente a hora -> retirar XXXXX
'out_of_stock':           Dummie enconde (somente 0 ou 1) -> One hot encode
'avg_delivery_time_days': sem outliers e distribuição normal -> min max scaler
'pid':                    valore duplicatidos, muitas labels -> usar para preencher valores nulos no price mas não vai para o modelo
'seller':                 Reunir undefined em "outros" e usar o frequency encode
'sub_category':           Unir com o 'category'
'fabrication_time':       sem outliers, distribuição uniforme -> min max scaler
'title':                  muitas categorias, sem concentrações -> frequency 
                          explorar mais o title
                          deletar nesse momento
'actual_price':           transformar em escala log

['_id',
 'average_rating',
 'number_of_reviews',
 'brand',
 'category',
 'crawled_at',
 'out_of_stock',
 'avg_delivery_time_days',
 'pid',
 'seller',
 'sub_category',
 'fabrication_time',
 'title',
 'actual_price']